In [1]:
import pandas as pd
import numpy as np
import random
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Semillas
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Config
MODEL_NAME = "bert-base-uncased"
TARGET = "gender"
CSV_PATH = "features_linguisticas_en_con_glove.csv"
COHEN_PATH = "../cohens_d_completo.csv"
COHEN_THRESHOLD = 0.14
BATCH_SIZE = 16
EPOCHS = 10
LR = 3e-5
FOLDS = 5
EARLY_STOPPING_PATIENCE = 4
MAX_LENGTH = 128

# Datos
df = pd.read_csv(CSV_PATH)
cohen_df = pd.read_csv(COHEN_PATH)

def normalize_gender(g):
    if pd.isnull(g): return np.nan
    g = str(g).strip().lower()
    if g in ["m", "male", "masculino"]: return "male"
    elif g in ["f", "female", "femenino"]: return "female"
    return np.nan

df["gender"] = df["gender"].apply(normalize_gender)
df = df[df["gender"].isin(["male", "female"])].copy()
df["label"] = df["gender"].map({"male": 0, "female": 1})

# Variables relevantes
selected_vars = cohen_df[
    (cohen_df["target"] == TARGET) &
    (cohen_df["cohens_d"].abs() > COHEN_THRESHOLD)
]["variable"].unique().tolist()

print(f"🔎 Usando {len(selected_vars)} variables GloVe con |d| > {COHEN_THRESHOLD}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
text_col = "clean_text"
df = df[[text_col, "label"] + selected_vars].dropna().reset_index(drop=True)

# Dataset
class MultiModalDataset(Dataset):
    def __init__(self, texts, nums, labels):
        self.texts = texts
        self.nums = nums
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        enc = tokenizer(self.texts[idx], truncation=True, padding="max_length", max_length=MAX_LENGTH, return_tensors="pt")
        item = {k: v.squeeze() for k, v in enc.items()}
        item["nums"] = torch.tensor(self.nums[idx], dtype=torch.float)
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Modelo
class TransformerWithTabular(nn.Module):
    def __init__(self, transformer_name, num_tabular_features):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(transformer_name)
        for name, param in self.transformer.named_parameters():
            if "encoder.layer.11" not in name:
                param.requires_grad = False

        self.tabular_net = nn.Sequential(
            nn.Linear(num_tabular_features, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 64)
        )
        self.classifier = nn.Sequential(
            nn.Linear(768 + 64, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 2)
        )

    def forward(self, input_ids, attention_mask, nums):
        out = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = out.last_hidden_state[:, 0]
        tabular_out = self.tabular_net(nums)
        combined = torch.cat([cls_output, tabular_out], dim=1)
        return self.classifier(combined)

# Preparación
texts = df[text_col].tolist()
features = df[selected_vars].values.astype(np.float32)
labels = df["label"].values

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
f1_scores = []

# Entrenamiento
for fold, (train_idx, val_idx) in enumerate(skf.split(texts, labels)):
    print(f"\n📦 Fold {fold+1}/{FOLDS}")

    X_train_texts = [texts[i] for i in train_idx]
    X_val_texts = [texts[i] for i in val_idx]
    y_train = labels[train_idx]
    y_val = labels[val_idx]
    # Oversampling manual de la clase minoritaria (female)
    X_train_feats_orig = features[train_idx]
    X_val_feats = features[val_idx]
    
    scaler = StandardScaler()
    X_train_feats_scaled = scaler.fit_transform(X_train_feats_orig)
    X_val_feats_scaled = scaler.transform(X_val_feats)
    
    X_train_texts_aug = list(X_train_texts)
    X_train_feats_aug = list(X_train_feats_scaled)
    y_train_aug = list(y_train)
    
    # Duplicar instancias de clase 'female' (label = 1)
    for i in range(len(y_train)):
        if y_train[i] == 1:
            X_train_texts_aug.append(X_train_texts[i])
            X_train_feats_aug.append(X_train_feats_scaled[i])
            y_train_aug.append(1)
    
    # Convertir a arrays
    X_train_feats_aug = np.array(X_train_feats_aug)
    y_train_aug = np.array(y_train_aug)

    scaler = StandardScaler()
    X_train_feats = scaler.fit_transform(features[train_idx])
    X_val_feats = scaler.transform(features[val_idx])

    train_ds = MultiModalDataset(X_train_texts, X_train_feats, y_train)
    val_ds = MultiModalDataset(X_val_texts, X_val_feats, y_val)
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

    model = TransformerWithTabular(MODEL_NAME, len(selected_vars)).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    scheduler = ReduceLROnPlateau(opt, mode='max', factor=0.5, patience=2)
    loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)

    best_f1 = 0
    patience = EARLY_STOPPING_PATIENCE

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for batch in tqdm(train_dl, desc=f"Fold {fold+1} Epoch {epoch+1}"):
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            nums = batch["nums"].to(device)
            lbls = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
            loss = loss_fn(outputs, lbls)

            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item()
        print(f"🔁 Epoch {epoch+1} loss: {total_loss/len(train_dl):.4f}")

        model.eval()
        all_preds, all_lbls = [], []
        with torch.no_grad():
            for batch in val_dl:
                input_ids = batch["input_ids"].to(device)
                attn = batch["attention_mask"].to(device)
                nums = batch["nums"].to(device)
                lbls = batch["labels"].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
                preds = torch.argmax(outputs, dim=1).cpu().numpy()
                all_preds.extend(preds)
                all_lbls.extend(lbls.cpu().numpy())

        f1 = f1_score(all_lbls, all_preds, average="macro")
        prec = precision_score(all_lbls, all_preds, average="macro", zero_division=0)
        rec = recall_score(all_lbls, all_preds, average="macro", zero_division=0)
        print(f"✅ F1: {f1:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}")

        scheduler.step(f1)

        if f1 > best_f1:
            best_f1 = f1
            patience = EARLY_STOPPING_PATIENCE
        else:
            patience -= 1
            if patience == 0:
                print("⏹️ Early stopping")
                break

    f1_scores.append(best_f1)
    print("📊 Matriz de confusión:")
    print(confusion_matrix(all_lbls, all_preds))
    from sklearn.metrics import classification_report
    print(classification_report(all_lbls, all_preds, target_names=["male", "female"]))

# Resultados finales
f1_avg = np.mean(f1_scores)
print("\n📊 F1 macro por fold:", [round(f, 4) for f in f1_scores])
print(f"🏁 F1 macro promedio final: {f1_avg:.4f}")


🔎 Usando 6 variables GloVe con |d| > 0.14

📦 Fold 1/5


2025-06-18 17:32:15.304735: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-18 17:32:15.315412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750260735.330751 3126711 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750260735.335332 3126711 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750260735.347714 3126711 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

🔁 Epoch 1 loss: 0.6941
✅ F1: 0.5385, Precision: 0.5402, Recall: 0.5410


Fold 1 Epoch 2: 100%|██████████| 48/48 [00:03<00:00, 13.56it/s]


🔁 Epoch 2 loss: 0.6872
✅ F1: 0.4805, Precision: 0.4990, Recall: 0.4992


...

🔁 Epoch 8 loss: 0.5726
✅ F1: 0.5862, Precision: 0.5924, Recall: 0.5867


Fold 5 Epoch 9: 100%|██████████| 48/48 [00:03<00:00, 13.43it/s]


🔁 Epoch 9 loss: 0.5341
✅ F1: 0.5412, Precision: 0.5469, Recall: 0.5475


Fold 5 Epoch 10: 100%|██████████| 48/48 [00:03<00:00, 13.49it/s]


🔁 Epoch 10 loss: 0.4993
✅ F1: 0.5459, Precision: 0.5658, Recall: 0.5636
📊 Matriz de confusión:
[[48 61]
 [26 57]]
              precision    recall  f1-score   support

        male       0.65      0.44      0.52       109
      female       0.48      0.69      0.57        83

    accuracy                           0.55       192
   macro avg       0.57      0.56      0.55       192
weighted avg       0.58      0.55      0.54       192


📊 F1 macro por fold: [0.5959, 0.6026, 0.5901, 0.5952, 0.5862]
🏁 F1 macro promedio final: 0.5940


In [2]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight

# Configuración
MODEL_NAME = "cardiffnlp/twitter-roberta-base"
TARGET = "ideology"
COHENS_D_FILE = "../cohens_d_completo.csv"
BATCH_SIZE = 16
EPOCHS = 10
LR = 1e-5
FOLDS = 5
COHEN_THRESHOLD = 0.10

# Leer CSVs
df = pd.read_csv("features_linguisticas_en_con_glove.csv")
cohen_df = pd.read_csv(COHENS_D_FILE)

# Filtrar target ideología
df = df[df["ideology"].isin(["izquierda", "derecha"])].copy()
df["label"] = df["ideology"].map({"izquierda": 0, "derecha": 1})

# Variables con d > 0.10 para ideology, excluyendo GloVe
selected_vars = cohen_df[
    (cohen_df["target"] == TARGET) & 
    (cohen_df["cohens_d"].abs() > COHEN_THRESHOLD) &
    (~cohen_df["variable"].str.startswith("XWE-"))
]["variable"].unique().tolist()

print(f"🔎 Variables seleccionadas para '{TARGET}' (|d| > {COHEN_THRESHOLD}): {selected_vars}")

# Limpiar y preparar
text_col = "clean_text"
df = df[[text_col, "label"] + selected_vars].dropna().reset_index(drop=True)

# Normalizar variables numéricas
scaler = StandardScaler()
df[selected_vars] = scaler.fit_transform(df[selected_vars])

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Dataset personalizado
class MultiModalDataset(Dataset):
    def __init__(self, texts, nums, labels):
        self.texts = texts
        self.nums = nums
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        enc = tokenizer(self.texts[idx], truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        item = {k: v.squeeze() for k, v in enc.items()}
        item["nums"] = torch.tensor(self.nums[idx], dtype=torch.float)
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Modelo
class TransformerWithTabular(nn.Module):
    def __init__(self, transformer_name, num_tabular_features):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(transformer_name)
        self.tabular_net = nn.Sequential(
            nn.Linear(num_tabular_features, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32)
        )
        self.classifier = nn.Sequential(
            nn.Linear(768 + 32, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 2)
        )

    def forward(self, input_ids, attention_mask, nums):
        out = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = out.last_hidden_state[:, 0]
        tabular_out = self.tabular_net(nums)
        combined = torch.cat([cls_output, tabular_out], dim=1)
        return self.classifier(combined)

# Preparar entrenamiento
texts = df[text_col].tolist()
features = df[selected_vars].values.astype(np.float32)
labels = df["label"].values

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
f1_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(texts, labels)):
    print(f"\n📦 Fold {fold+1}/{FOLDS}")
    train_ds = MultiModalDataset([texts[i] for i in train_idx], features[train_idx], labels[train_idx])
    val_ds = MultiModalDataset([texts[i] for i in val_idx], features[val_idx], labels[val_idx])
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

    model = TransformerWithTabular(MODEL_NAME, len(selected_vars)).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)

    # Entrenamiento
    model.train()
    for epoch in range(EPOCHS):
        total_loss = 0
        for batch in tqdm(train_dl, desc=f"Fold {fold+1} Epoch {epoch+1}"):
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            nums = batch["nums"].to(device)
            lbls = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
            loss = loss_fn(outputs, lbls)

            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item()
        print(f"🔁 Epoch {epoch+1} loss: {total_loss/len(train_dl):.4f}")

    # Evaluación
    model.eval()
    all_preds, all_lbls = [], []
    with torch.no_grad():
        for batch in val_dl:
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            nums = batch["nums"].to(device)
            lbls = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_lbls.extend(lbls.cpu().numpy())

    f1 = f1_score(all_lbls, all_preds, average="macro")
    f1_scores.append(f1)
    print(f"✅ F1 Fold {fold+1}: {f1:.4f}")
    from sklearn.metrics import classification_report
    print(classification_report(all_lbls, all_preds, target_names=["izquierda", "derecha"]))

# Final
f1_avg = np.mean(f1_scores)
print("\n📊 F1 macro por fold:", [round(f, 4) for f in f1_scores])
print(f"🏁 F1 macro promedio final: {f1_avg:.4f}")


🔎 Variables seleccionadas para 'ideology' (|d| > 0.1): ['Xwords', 'Xtwice', 'Xmax_length', 'Xcharacter', 'Xcapital', 'Xword_par', 'Xchar_par', 'Xdet', 'Xprep', 'Xpronouns', 'Xmentions']

📦 Fold 1/5


Fold 1 Epoch 1: 100%|██████████| 48/48 [00:09<00:00,  5.19it/s]


🔁 Epoch 1 loss: 0.6950


Fold 1 Epoch 2: 100%|██████████| 48/48 [00:09<00:00,  5.20it/s]


🔁 Epoch 2 loss: 0.6693


...

🔁 Epoch 8 loss: 0.0942


Fold 5 Epoch 9: 100%|██████████| 48/48 [00:04<00:00, 10.43it/s]


🔁 Epoch 9 loss: 0.0624


Fold 5 Epoch 10: 100%|██████████| 48/48 [00:04<00:00, 10.39it/s]


🔁 Epoch 10 loss: 0.0455
✅ F1 Fold 5: 0.6452
              precision    recall  f1-score   support

       young       0.72      0.85      0.78       122
         old       0.62      0.43      0.51        70

    accuracy                           0.70       192
   macro avg       0.67      0.64      0.65       192
weighted avg       0.69      0.70      0.68       192


📊 F1 macro por fold: [0.6129, 0.7032, 0.647, 0.6653, 0.6452]
🏁 F1 macro promedio final: 0.6547


In [3]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight

# Configuración
MODEL_NAME = "cardiffnlp/twitter-roberta-base"
TARGET = "age_group"
COHENS_D_FILE = "../cohens_d_completo.csv"
BATCH_SIZE = 16
EPOCHS = 10
LR = 1e-5
FOLDS = 5
COHEN_THRESHOLD = 0.1

# Leer CSVs
df = pd.read_csv("features_linguisticas_en_con_glove.csv")
cohen_df = pd.read_csv(COHENS_D_FILE)

# Filtrar target ideología
df = df[df["ideology"].isin(["izquierda", "derecha"])].copy()
df["label"] = df["ideology"].map({"izquierda": 0, "derecha": 1})

# Variables con d > 0.10 para ideology, excluyendo GloVe
selected_vars = cohen_df[
    (cohen_df["target"] == TARGET) & 
    (cohen_df["cohens_d"].abs() > COHEN_THRESHOLD) &
    (~cohen_df["variable"].str.startswith("XWE-"))
]["variable"].unique().tolist()

print(f"🔎 Variables seleccionadas para '{TARGET}' (|d| > {COHEN_THRESHOLD}): {selected_vars}")

# Limpiar y preparar
text_col = "clean_text"
df = df[[text_col, "label"] + selected_vars].dropna().reset_index(drop=True)

# Normalizar variables numéricas
scaler = StandardScaler()
df[selected_vars] = scaler.fit_transform(df[selected_vars])

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Dataset personalizado
class MultiModalDataset(Dataset):
    def __init__(self, texts, nums, labels):
        self.texts = texts
        self.nums = nums
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        enc = tokenizer(self.texts[idx], truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        item = {k: v.squeeze() for k, v in enc.items()}
        item["nums"] = torch.tensor(self.nums[idx], dtype=torch.float)
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Modelo
class TransformerWithTabular(nn.Module):
    def __init__(self, transformer_name, num_tabular_features):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(transformer_name)
        self.tabular_net = nn.Sequential(
            nn.Linear(num_tabular_features, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32)
        )
        self.classifier = nn.Sequential(
            nn.Linear(768 + 32, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 2)
        )

    def forward(self, input_ids, attention_mask, nums):
        out = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = out.last_hidden_state[:, 0]
        tabular_out = self.tabular_net(nums)
        combined = torch.cat([cls_output, tabular_out], dim=1)
        return self.classifier(combined)

# Preparar entrenamiento
texts = df[text_col].tolist()
features = df[selected_vars].values.astype(np.float32)
labels = df["label"].values

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
f1_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(texts, labels)):
    print(f"\n📦 Fold {fold+1}/{FOLDS}")
    train_ds = MultiModalDataset([texts[i] for i in train_idx], features[train_idx], labels[train_idx])
    val_ds = MultiModalDataset([texts[i] for i in val_idx], features[val_idx], labels[val_idx])
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

    model = TransformerWithTabular(MODEL_NAME, len(selected_vars)).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)

    # Entrenamiento
    model.train()
    for epoch in range(EPOCHS):
        total_loss = 0
        for batch in tqdm(train_dl, desc=f"Fold {fold+1} Epoch {epoch+1}"):
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            nums = batch["nums"].to(device)
            lbls = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
            loss = loss_fn(outputs, lbls)

            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item()
        print(f"🔁 Epoch {epoch+1} loss: {total_loss/len(train_dl):.4f}")

    # Evaluación
    model.eval()
    all_preds, all_lbls = [], []
    with torch.no_grad():
        for batch in val_dl:
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            nums = batch["nums"].to(device)
            lbls = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_lbls.extend(lbls.cpu().numpy())

    f1 = f1_score(all_lbls, all_preds, average="macro")
    f1_scores.append(f1)
    print(f"✅ F1 Fold {fold+1}: {f1:.4f}")
    from sklearn.metrics import classification_report
    print(classification_report(all_lbls, all_preds, target_names=["izquierda", "derecha"]))

# Final
f1_avg = np.mean(f1_scores)
print("\n📊 F1 macro por fold:", [round(f, 4) for f in f1_scores])
print(f"🏁 F1 macro promedio final: {f1_avg:.4f}")


🔎 Variables seleccionadas para 'age_group' (|d| > 0.1): ['Xwords', 'Xunique', 'Xmax_length', 'Xlength_3', 'Xstop', 'Xcharacter', 'Xcapital', 'Xpunctuation', 'Xword_par', 'Xchar_par', 'Xdet', 'Xprep', 'Xsing', 'Xplural', 'Xadv', 'Xnouns', 'Xconj', 'Xpast', 'Xhashtag', 'Xurl']

📦 Fold 1/5


2025-06-18 22:21:53.096953: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-18 22:21:53.107534: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750278113.122461 3236863 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750278113.126936 3236863 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750278113.138959 3236863 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

🔁 Epoch 1 loss: 0.6955


Fold 1 Epoch 2: 100%|██████████| 48/48 [00:09<00:00,  4.83it/s]


🔁 Epoch 2 loss: 0.6701


...

🔁 Epoch 8 loss: 0.0946


Fold 5 Epoch 9: 100%|██████████| 48/48 [00:10<00:00,  4.79it/s]


🔁 Epoch 9 loss: 0.0560


Fold 5 Epoch 10: 100%|██████████| 48/48 [00:09<00:00,  4.83it/s]


🔁 Epoch 10 loss: 0.0330
✅ F1 Fold 5: 0.6501
              precision    recall  f1-score   support

   izquierda       0.73      0.82      0.77       122
     derecha       0.60      0.47      0.53        70

    accuracy                           0.69       192
   macro avg       0.66      0.65      0.65       192
weighted avg       0.68      0.69      0.68       192


📊 F1 macro por fold: [0.6326, 0.6711, 0.5866, 0.672, 0.6501]
🏁 F1 macro promedio final: 0.6425
